In [182]:
directory = 'tw/price'
list_ = os.listdir(directory)
df_list = [pd.read_parquet(os.path.join(directory, i)) for i in list_]
df_all = pd.concat(df_list)
df_all['da1'] = df_all['da'].apply(lambda x: f"{int(x[:3])+1911}-{x[4:6]}-{x[7:9]}")
df = df_all.drop_duplicates(subset=['ticker', 'da1'])
df['da_check'] = df['da1'].astype(str).apply(lambda x: int(x[:4]))
df_final = df[df['da_check'].isin([i for i in range(2018, 2026)])]

for values in ['cl', 'op', 'vol(volume)', 'vol(turnover)']:
    df_pivot = df_final.pivot(values=values, columns='ticker', index='da1')
    df_pivot.replace("--", None, inplace=True)
    df_pivot.replace("-", None, inplace=True)
    df_pivot.ffill(inplace=True)
    df_pivot = df_pivot.loc['2018-01-01':(datetime.now()+timedelta(days=1)).strftime("%Y-%m-%d")]
    df_pivot.sort_index(ascending=True, inplace=True)
    df_pivot = df_pivot.apply(lambda x: x.str.replace(',', '').astype(float))
    df_pivot.dropna(how='all', axis=1, inplace=True)
    df_pivot.to_parquet(f"tw/pdata/{values}.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}.parquet successful")
    df_pivot_pct = df_pivot.pct_change().dropna()
    df_pivot.to_parquet(f"tw/pdata/{values}_pct.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}_pct.parquet successful")
    


store pdata of values cl to tw/pdata/cl.parquet successful
store pdata of values cl to tw/pdata/cl_pct.parquet successful
store pdata of values op to tw/pdata/op.parquet successful
store pdata of values op to tw/pdata/op_pct.parquet successful
store pdata of values vol(volume) to tw/pdata/vol(volume).parquet successful
store pdata of values vol(volume) to tw/pdata/vol(volume)_pct.parquet successful
store pdata of values vol(turnover) to tw/pdata/vol(turnover).parquet successful
store pdata of values vol(turnover) to tw/pdata/vol(turnover)_pct.parquet successful


### 富櫃50 TPEX

In [ ]:

import requests
list_list = []
for year in range(113, 102, -1):
    limit_month = 8 if year == 113 else 13
    for index, month in enumerate(range(1, limit_month)):
        month = f"0{month}" if month < 10 else month
        url_gretai50 = f"https://www.tpex.org.tw/web/stock/iNdex_info/gretai50/inxhis/rihisqry_result.php?l=zh-tw&d={year}/{month}/01&_=1720688879318"
        response = requests.get(url_gretai50)
        data = response.json()['aaData']
        list_list.append(data)
list_ = sum(list_list, [])
df = pd.DataFrame(list_)
df.to_parquet('../db/tw/ind/gretai50.parquet')

### 財報, selenium

In [ ]:
'''
cd /Users/test/risksoft/TS/db/chromedriver_mac/chromedriver-mac-x64/
chmod +x chromedriver

or

chmod +x /Users/test/risksoft/TS/db/chromedriver_mac/chromedriver-mac-x64/chromedriver

so that auto generated chromedriver can be reached and executed
'''

In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 初始化 Selenium WebDriver
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")
service = Service(executable_path=obj.chromedriver_path)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import os
import json
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
os.chdir("../db/")

In [2]:
import os
os.getcwd()

'/Users/test/risksoft/TS/db'

In [2]:
with open("../db/tw/symbol/symbol_4.json", "r") as f:
    tw_symbol_4 = json.load(f)
list_tw_stock = [key for key, value in tw_symbol_4.items() if value=="TW"]

In [4]:
arrs = []
try_ = 3
sleep_ = 3
season_dict = {
1: '03-31',
2: "06-30",
3: "09-30",
4: "12-31",
}
slices = {
    'df_current': slice(5, 15),
    'df_noncurrent': slice(18, 25),
    'df_total_asset': 26,
    'df_currentlia': slice(28, 36),
    'df_noncurrentlia': slice(38, 43),
    'df_total_lia': 44,
    'df_equity': slice(-5, -3)
}
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model, useful for running on servers
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems

# Set up the Chrome WebDriver with the specified options
driver = webdriver.Chrome(options=chrome_options)
url = 'https://mops.twse.com.tw/mops/web/t164sb03'
driver.get(url)
wait = WebDriverWait(driver, 20)
time.sleep(2)
select_element_isnew = wait.until(EC.presence_of_element_located((By.ID, 'isnew')))
select_isnew = Select(select_element_isnew)
company_code_input = wait.until(EC.presence_of_element_located((By.ID, 'co_id')))
year_input = driver.find_element(By.ID, 'year')
select_element_Season = wait.until(EC.presence_of_element_located((By.ID, 'season')))
select_season = Select(select_element_Season)
search_button = driver.find_element(By.CSS_SELECTOR, 'input[type="button"][value=" 查詢 "]')
for year in [111, 109]:
    if year != 111:
        year_input.clear()
    for season in [1, 2, 3, 4]:
        for ticker in tqdm(list_tw_stock, desc=f"{year}, {season}"):
            wait = WebDriverWait(driver, 20)
            select_isnew.select_by_visible_text('歷史資料')

            company_code_input.clear()
            company_code_input.send_keys(ticker)

            year_input.send_keys(str(year))

            select_season.select_by_visible_text(str(season))
            search_button.click()

            wait.until(EC.presence_of_element_located((By.ID, 'table01')))
            time.sleep(2)
            data = []
            df = None
            for i in range(try_):
                try:
                    table = driver.find_element(By.CLASS_NAME, 'hasBorder')
                    rows = table.find_elements(By.TAG_NAME, "tr")
                    for row in rows:
                        cols = row.find_elements(By.TAG_NAME, 'td')
                        cols = [col.text for col in cols]
                        data.append(cols)
                        df = pd.DataFrame(data)
                    break
                except Exception as e:
                    # financial statements not exist
                    break
            if df is None:
                continue
            try:
                df_sp = df.iloc[:, [0, 1, 2, -2, -1]]
                dfs = {name: df_sp.iloc[s] if isinstance(s, slice) else df_sp.iloc[[s], :] for name, s in slices.items()}
                df_concat = pd.concat(dfs.values())
                df_concat.insert(3, "season_da1", f"{1911+year}-{season_dict[season]}")
                df_concat.insert(4, "ticker", ticker)
                df_concat.insert(5, "name2", df_concat.iloc[:, 0])
                df_concat.insert(8, "season_da2", f"{1911+year-1}-{season_dict[season]}")
                df_concat.insert(9, "ticker1", ticker)
                array_f = df_concat.values.reshape(int(df_concat.shape[0]*2), int(df_concat.shape[1]/2))
                arrs.append(array_f)
            except IndexError:
                print(ticker, year, season)
                continue

111, 1:  14%|█▍        | 146/1030 [14:22<1:14:50,  5.08s/it]

5871 111 1


111, 1:  26%|██▌       | 268/1030 [27:20<1:05:30,  5.16s/it]

2850 111 1


111, 1:  40%|███▉      | 407/1030 [42:07<54:57,  5.29s/it]  

...

109, 1:  41%|████      | 422/1030 [47:03<1:08:42,  6.78s/it]

1438 109 1


109, 1:  44%|████▍     | 451/1030 [50:17<1:01:15,  6.35s/it]

2882 109 1


109, 1:  49%|████▉     | 505/1030 [57:31<1:06:12,  7.57s/it]

In [36]:
df_long_format = pd.DataFrame(sum([array_f.tolist() for array_f in arrs], []))


,0,1,2,3,4
0,現金及約當現金,"1,151,589,646",28.84,2022-03-31,2330
1,現金及約當現金,"664,727,857",22.77,2021-03-31,2330
2,透過損益按公允價值衡量之金融資產－流動,"758,452",0.02,2022-03-31,2330
3,透過損益按公允價值衡量之金融資產－流動,"26,897",0.00,2021-03-31,2330
4,透過其他綜合損益按公允價值衡量之金融資產－流動,"120,995,765",3.03,2022-03-31,2330
...,...,...,...,...,...
1083,遞延所得稅負債,"18,261,509",0.55,2019-12-31,2317
1084,權益總額,"1,474,146,409",40.12,2020-12-31,2317
1085,權益總額,"1,399,625,417",42.15,2019-12-31,2317
1086,負債及權益總計,"3,674,275,590",100.00,2020-12-31,2317
